In [120]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [121]:
#Import movies csv
movies_df = pd.read_csv('../data/movies.csv')
# movies_df.head()
# len(movies_df)

In [122]:
#Import ratings csv
ratings_df = pd.read_csv('../data/ratings.csv', parse_dates=['timestamp'])
ratings_df = ratings_df.drop(columns=['timestamp'])
# ratings_df.head()

In [123]:
#Combine 'movies' and 'ratings' dataframes and drop timestamp column
combined_df = pd.merge(ratings_df, movies_df, how='right', on='movieId')
# combined_df = combined_df.drop(columns=['timestamp'])
# combined_df

In [124]:
#Extract a list of individual genres from 'genres' column in df
genres_list = []
unique_genres = []
for genre in combined_df.genres.unique():
    genres_list.append(genre)
for x in range(len(genres_list)):
    genres_list[x] = genres_list[x].split('|')
for x in genres_list:
    for genre in x:
        if genre not in unique_genres:
            unique_genres.append(genre)
# unique_genres

In [125]:
all_genres = []
for x in movies_df['genres']:
    all_genres.append(x)
# all_genres

In [126]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_model = TfidfVectorizer(analyzer='word',stop_words='english')
tfidf_values = tfidf_model.fit_transform(movies_df['genres'])
# tfidf_values.shape

In [127]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_values, tfidf_values)
# cosine_sim.shape

In [128]:
movie_titles = np.array(movies_df['title'])
def recommendation_engine_1(movie_title):
    for x in range(len(movie_titles)):
        if movie_titles[x] == movie_title:
            index = x
            similarity = list(enumerate(cosine_sim[index]))
    similarity = sorted(similarity, key=lambda x: x[1], reverse=True)
    similarity = similarity[1:11]
    movie_indices = [i[0] for i in similarity]
    return movie_titles[movie_indices] 

In [129]:
# print(f"Because you liked Jumanji, we recommend the following movies: \n{recommendation_engine_1('Jumanji (1995)')}")

<!-- from sklearn.metrics.pairwise import pairwise_distances
collab_df = ratings_df.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
train_data, test_data = train_test_split(collab_df, test_size=0.2)
train_data_matrix = train_data.to_numpy()
test_data_matrix = test_data.to_numpy()
user_correlation = 1 - pairwise_distances(train_data, metric='correlation')
user_correlation[np.isnan(user_correlation)] = 0
collab_matrix = collab_df.to_numpy()
user_ratings_mean = np.mean(collab_matrix, axis = 1)
normalized_matrix = collab_matrix - user_ratings_mean.reshape(-1, 1)
item_correlation = 1 - pairwise_distances(train_data_matrix.T, metric='correlation')
item_correlation[np.isnan(item_correlation)] = 0
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # Use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred
from sklearn.metrics import mean_squared_error
from math import sqrt

# Function to calculate RMSE
def rmse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))
# Predict ratings on the training data with both similarity score
user_prediction = predict(train_data_matrix, user_correlation, type='user')
item_prediction = predict(train_data_matrix, item_correlation, type='item')
# RMSE on the train data
print('User-based CF RMSE: ' + str(rmse(user_prediction, train_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, train_data_matrix))) -->

In [130]:
#Turn df into matrix with users as rows and movies as columns
collab_df = ratings_df.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
collab_matrix = collab_df.to_numpy()
# print(collab_matrix)
#Normalize data
user_ratings_mean = np.mean(collab_matrix, axis = 1)
normalized_matrix = collab_matrix - user_ratings_mean.reshape(-1, 1)
# normalized_matrix

In [131]:

# Item Similarity Matrix
# item_correlation = 1 - pairwise_distances(train_data_matrix.T, metric='correlation')
# item_correlation[np.isnan(item_correlation)] = 0
# def predict(ratings, similarity, type='user'):
#     if type == 'user':
#         mean_user_rating = ratings.mean(axis=1)
#         # Use np.newaxis so that mean_user_rating has same format as ratings
#         ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
#         pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
#     elif type == 'item':
#         pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
#     return pred

In [132]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(normalized_matrix, k = 200)
sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
full_predictions_df = pd.DataFrame(all_user_predicted_ratings, columns = collab_df.columns)
full_predictions_df

movieId,1,2,3,4,5,6,7,8,9,10,...,144482,144656,144976,146344,146656,146684,146878,148238,148626,149532
0,0.252299,-0.010088,0.092040,0.192354,-0.024426,1.204934,-0.015283,0.129069,0.121097,1.111297,...,-0.002755,-0.022120,-0.001751,-0.009376,-0.031698,-0.015677,-0.006942,-0.017872,-0.009370,-0.026368
1,4.344047,0.336675,2.034144,-0.145177,2.408448,1.553709,1.451530,-0.019907,0.387475,0.168440,...,-0.004785,-0.011620,0.002777,-0.004533,-0.022193,-0.005621,-0.001670,-0.009257,-0.005303,-0.013982
2,-0.047223,0.393745,0.510795,-0.004261,1.027274,-0.218834,1.406311,-0.105666,-0.218101,0.882176,...,-0.002124,-0.009580,0.012006,-0.008204,-0.003462,-0.039843,-0.027290,-0.009122,-0.016422,-0.010039
3,0.360930,0.540959,-0.489033,0.135677,-0.073150,-0.667607,-0.062832,-0.002981,0.194092,-0.107255,...,0.001363,0.013956,-0.000401,0.008675,0.011834,-0.016573,-0.009745,0.012196,0.034279,0.015716
4,3.006196,0.547924,-0.149652,0.039269,-0.088289,0.110227,-0.328344,0.043419,-0.091429,-0.002749,...,0.020532,0.004927,-0.006255,0.004344,0.027355,-0.006506,-0.002729,0.004733,-0.010034,0.005122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,0.204212,0.554004,-0.120526,-0.031133,0.523929,-0.080739,-0.103719,-0.036150,-0.097324,-0.081746,...,-0.008146,-0.016703,0.019206,-0.009123,-0.026350,0.112957,0.070967,-0.014176,0.002973,-0.019230
664,-0.278322,-0.359287,0.518868,0.014129,0.680634,0.219206,0.135696,-0.040995,-0.088975,-0.280317,...,-0.018149,0.011731,0.008699,0.008711,-0.019722,-0.014468,-0.007286,0.010724,0.007465,0.012737
665,-0.268481,0.257445,-0.174872,0.029480,0.165592,-0.071909,-0.074098,-0.054978,-0.039878,0.218998,...,-0.010088,-0.004009,-0.000906,-0.002108,-0.017393,-0.002462,-0.001379,-0.003375,0.012561,-0.004642
666,1.056872,-0.425884,-0.360974,0.064453,-0.062715,0.221078,0.045574,-0.013326,0.166967,0.297952,...,-0.014556,-0.003292,0.008689,-0.003184,-0.018612,-0.035541,-0.023353,-0.003256,0.075088,-0.003328


In [133]:
def recommendation_engine_2(full_predictions_df, userId, movies_df, ratings_df):
    
    user_predictions_df = full_predictions_df.iloc[userId - 1].sort_values(ascending=False)
    user_ratings_df = ratings_df[ratings_df.userId == (userId)]
    user_full = (user_ratings_df.merge(movies_df, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False))
    top_rated_movies = user_full['title'].to_list()
    recommended_movie_ids = user_predictions_df.index
    already_rated_ids = user_ratings_df['movieId'].to_list()
    final_recommendations = []
    for x in recommended_movie_ids:
        if x not in already_rated_ids:
            final_recommendations.append(movies_df.loc[movies_df['movieId']==x]['title'])
    
    return f"UserID {userId} ",top_rated_movies[0:10], final_recommendations[0:5]

In [134]:
recommendation_engine_2(X_test, 1, movies_df, ratings_df)

C:\Users\erikt\anaconda3\envs\PythonData\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
C:\Users\erikt\anaconda3\envs\PythonData\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


('UserID 1 ',
 ['Casablanca (1942)',
  'Monty Python and the Holy Grail (1975)',
  'Scarface (1983)',
  'Godfather: Part II, The (1974)',
  'Fight Club (1999)',
  'Silence of the Lambs, The (1991)',
  'Spartacus (1960)',
  'Patriot Games (1992)',
  'Godfather, The (1972)',
  'Goodfellas (1990)'],
 [Series([], Name: title, dtype: object),
  Series([], Name: title, dtype: object)])

In [52]:
from numpy import random
random_user = np.random.randint(0,high=668)
recommendation_engine_2(full_predictions_df, random_user, movies_df, ratings_df)

('UserID 273 ',
 ['Shawshank Redemption, The (1994)',
  'Ghost (1990)',
  'Rock, The (1996)',
  'K-PAX (2001)',
  'Pulp Fiction (1994)',
  '15 Minutes (2001)',
  'Sixth Sense, The (1999)',
  'Gladiator (2000)',
  'Negotiator, The (1998)',
  'Lives of Others, The (Das leben der Anderen) (2006)'],
 [3344    Scarface (1983)
  Name: title, dtype: object,
  257    Outbreak (1995)
  Name: title, dtype: object,
  309    Clear and Present Danger (1994)
  Name: title, dtype: object,
  1289    Hunt for Red October, The (1990)
  Name: title, dtype: object,
  3169    Finding Forrester (2000)
  Name: title, dtype: object])

In [119]:
# Import libraries from Surprise package
# !pip install surprise
# from surprise import Reader, Dataset, SVD, evaluate
# from sklearn.decomposition import TruncatedSVD
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import GridSearchCV
# data = ratings_df[['userId', 'movieId', 'rating']]
# data.to_numpy()
# data.split(n_folds=5)
# svd = TruncatedSVD()
# GridSearchCV(svd,data)

AttributeError: 'DataFrame' object has no attribute 'split'